In [1]:
import sys
import os
import base64
from IPython.display import display, Image

# 添加 'src' 目录到 Python 路径
sys.path.append(os.path.abspath(os.path.join('..')))

# 确保我们导入的是新版本
from src.bank_ds_agent.tools.code_executor import SandboxJupyterExecutor

print("正在创建沙箱化 Jupyter 执行器实例...")
# 这一步现在会启动一个 Docker 容器
executor = SandboxJupyterExecutor()
print("实例创建成功。")

正在创建沙箱化 Jupyter 执行器实例...
正在初始化 Docker 沙箱 (使用端口映射)...
正在从镜像启动容器: bank-agent-kernel
等待内核在 C:\Users\User\OneDrive\Desktop\Agentic AI\Agentic-AI-Data-Scientist-Agent-in-Banking-Sector\kernel.json 创建并写入连接文件...
kernel.json 已找到并成功解析。
原始 IP: 0.0.0.0
修正后 IP: 127.0.0.1
正在从 *修正后* 的配置加载 KernelClient...
!! 启动沙箱时出错: Kernel died before replying to kernel_info

--- [DOCKER 容器日志] ---
NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /app/kernel.json

---------------------------


--- [沙箱Jupyter] 正在关闭 ---
已删除 C:\Users\User\OneDrive\Desktop\Agentic AI\Agentic-AI-Data-Scientist-Agent-in-Banking-Sector\kernel.json
Jupyter 客户端已停止。
正在停止和移除容器: 16c51301db6d...
容器已停止。


RuntimeError: Kernel died before replying to kernel_info

In [ ]:
# 1. 安装库
result_install = executor.run_code("import sys; !{sys.executable} -m pip install pandas matplotlib")
print(result_install)

# 2. 定义一个变量 (测试状态)
result_define = executor.run_code("import pandas as pd; df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})")
print("\n--- 定义变量 ---")
print(result_define)

In [ ]:
# 3. 在 *新的* 调用中访问该变量
# 如果内核是有状态的，这将成功。
result_use = executor.run_code("print(df.head())")

print("\n--- 使用变量 (stdout) ---")
print(result_use['stdout'])

In [ ]:
# 4. 生成一个图表
plot_code = """
import matplotlib.pyplot as plt
import io
import base64

# 这一行告诉 matplotlib 将图表直接输出到 notebook
%matplotlib inline

x = [1, 2, 3, 4]
y = [10, 20, 25, 30]
plt.figure()
plt.plot(x, y)
plt.title('My Test Plot')
plt.show()
"""

plot_result = executor.run_code(plot_code)

print("\n--- 绘图结果 ---")
# 显示捕获的图像
if plot_result['images']:
    print("成功捕获图像！正在显示：")
    img_data = base64.b64decode(plot_result['images'][0])
    display(Image(data=img_data))
else:
    print("未能捕获图像。")
    print(f"Stdout: {plot_result['stdout']}")
    print(f"Stderr: {plot_result['stderr']}")

In [ ]:
# 5. 测试一个运行时错误
error_code = "print(df['non_existent_column'])"
error_result = executor.run_code(error_code)

print("\n--- 错误捕获 (stderr) ---")
print(error_result['stderr'])

In [ ]:
# 6. (重要!) 完成后关闭内核
executor.close()